#  [모듈 4.5] 모델 레지스트리로 부터 모델 배포 및  람다 스텝 개발 (SageMaker Lambda Step)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 1. 모델 레지스트리를 통한 모델 배포
    - (1) 모델 레지스트리에서 모델 등록 확인
    - (2) 모델 버전 승인 상태 변경
    - (3) 모델 배포 
- 2. 람다 스텝 개요
    - 위의 `"모델 레지스트리를 통한 모델 배포"` 를 람다 스텝 및 모델 생성 스텝을 통하여 구현 합니다.
- 3. 리소스 정리

    
---

# 1. 모델 레지스트리를 통한 모델 배포

---

`import ` 시마다 원본 소스에서 재로딩을 설정함. (다른 소스 파일을 수정후에 디버깅하기에 편함)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.client("sagemaker")

%store -r 
# 노트북에 저장되어 있는 변수를 보기 위해서는 주석을 제거하고 실행하시면 됩니다.
# %store  

## (1) 모델 레지스트리에서 모델 등록 확인
위에서 등록한 모델 그룹 이름을 통해서 어떤 모델이 등록되었는지를 확인 합니다.
- 등록된 모델 버전에 대한 보기 --> [모델 버전의 세부 정보 보기](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-registry-details.html)

In [3]:
import boto3
sm_client = boto3.client('sagemaker')

# 위에서 생성한 model_package_group_name 을 인자로 제공 합니다.
response = sm_client.list_model_packages(ModelPackageGroupName= model_package_group_name)
response

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase01',
   'ModelPackageVersion': 2,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase01/2',
   'CreationTime': datetime.datetime(2021, 8, 25, 4, 57, 13, 457000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase01',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase01/1',
   'CreationTime': datetime.datetime(2021, 8, 25, 4, 54, 13, 468000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'}],
 'ResponseMetadata': {'RequestId': '52cf11b0-6fcd-489b-bf77-9c6caa3512a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '52cf11b0-6fcd-489b-bf77

#### 등록된 모델 버전의 상세 정보 확인

In [4]:
ModelPackageArn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
sm_client.describe_model_package(ModelPackageName=ModelPackageArn)

{'ModelPackageGroupName': 'sagemaker-pipeline-step-by-step-phase01',
 'ModelPackageVersion': 2,
 'ModelPackageArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model-package/sagemaker-pipeline-step-by-step-phase01/2',
 'CreationTime': datetime.datetime(2021, 8, 25, 4, 57, 13, 457000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
    'ImageDigest': 'sha256:04889b02181f14632e19ef6c2a7d74bfe699ff4c7f44669a78834bc90b77fe5a',
    'ModelDataUrl': 's3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/training_jobs/1hzhiaghayc0-HPTunin-jBvIJGSXWm-001-493861ef/output/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m5.large'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.t2.medium', 'ml.m5.large'],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStat

## (2) Model 승인 상태 변경

In [5]:
model_package_update_input_dict = {
    "ModelPackageArn" : ModelPackageArn,
    "ModelApprovalStatus" : "Approved"
}
model_package_update_response = sm_client.update_model_package(**model_package_update_input_dict)
response = sm_client.describe_model_package(ModelPackageName=ModelPackageArn)

In [6]:
image_uri_approved = response["InferenceSpecification"]["Containers"][0]["Image"]
ModelDataUrl_approved = response["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]
print("image_uri_approved: ", image_uri_approved)
print("ModelDataUrl_approved: ", ModelDataUrl_approved)

image_uri_approved:  366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3
ModelDataUrl_approved:  s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/training_jobs/1hzhiaghayc0-HPTunin-jBvIJGSXWm-001-493861ef/output/model.tar.gz


## (3) 모델을  앤드포인트 배포

In [7]:
%%time 

from sagemaker import ModelPackage

model = ModelPackage(role=role, 
                     model_package_arn=ModelPackageArn, 
                     sagemaker_session=sagemaker_session)
_ = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


---------------!CPU times: user 234 ms, sys: 15.3 ms, total: 249 ms
Wall time: 7min 32s


배포된 모델을  삭제 합니다.

In [8]:
predictor = sagemaker.predictor.Predictor(
    endpoint_name= model.endpoint_name,
    sagemaker_session= sagemaker_session,
)

predictor.delete_endpoint()

# 2. 람다 스텝 개요

- 세이지 메이커 모델 빌딩 파이프라인 에서 람다 스텝을 (2021.08) 사용할 수 있습니다. 람다 함수에 머신러닝 워크 플로우에 적당한 단계를 구현하여 사용할 수 있습니다.
- 여기서는 예시로써, 모델 레지스트리의 모델 패키지 그룹에 있는 최신 모델의 승인 상태를 변경하고, 배포까지 하는 것을 람다 스텝을 이용하여 구현 합니다.
- 람다 스텝 사용 법
    - LambdaStep을 정의할 때 SageMaker Lambda 도우미 클래스는 Lambda 함수를 생성하기 위한 도우미 함수를 제공합니다.사용자는 lambda_func 인수를 사용하여 이미 배포된 Lambda 함수에 함수 ARN을 제공하거나 Lambda 클래스를 사용하여 Lambda 함수에 대한 스크립트, 함수 이름 및 역할을 제공하여 Lambda 함수를 생성할 수 있습니다.

    - 입력을 Lambda에 전달할 때 inputs 인수를 사용할 수 있으며 Lambda 함수의 핸들러 내에서 event 인수를 사용하여 입력을 검색할 수 있습니다.

    - Lambda 함수의 딕셔너리 응답은 outputs 인수에 제공된 lambdaOutput 객체를 통해 구문 분석됩니다.LambdaOutput 의 output_name은 람다의 리턴 딕셔너리에 있는 딕셔너리 키에 해당합니다.


- 참고
    - 개발자 가이드의 람다 단계 참고 --> [람다 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-lambda)


#### 람다 함수 정의
- 여기서는 2개의 람다 함수를 정의 했습니다.
    - (1) iam_change_model_approval.py
        - 모델 레지스트리에서 해당 모델 패키지 그룹을 조회하고, 가장 최신 버전의 모델에 대해서 '모델 승인 상태 변경' 을 합니다.
    - (2) iam_create_endpoint.py
        - 입력으로 세이지 메이커 모델, 앤드 포인트 컨피그 및 앤드 포인트 이름을 받아서, 앤드포인트를 생성 함.
        

- 예시로써 첫번째 람다 함수를 정의한 것의 예제 입니다.


In [9]:
!pygmentize src/iam_change_model_approval.py

import json
import boto3


def lambda_handler(event, context):
    """
    모델 레지스트리에서 최신 버전의 모델 승인 상태를 변경하는 람다 함수.
    """
    
    try:
        sm_client = boto3.client("sagemaker")

        ##############################################
        # 람다 함수는 두개의 입력 인자를 event 개체를 통해서 받습니다.
        # 모델 패키지 이름과 모델 승인 유형을 받습니다.
        ##############################################   
        
        model_package_group_name = event["model_package_group_name"]
        ModelApprovalStatus = event["ModelApprovalStatus"]        
        print("model_package_group_name: \n", model_package_group_name)
        print("ModelApprovalStatus: \n", ModelApprovalStatus)        

        
        # 해당 모델 패키지에서 가장 최근의 버전의 모델을 가져옵니다.
        response = sm_client.list_model_packages(ModelPackageGroupName= model_package_group_name)
        ModelPackageArn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
        sm_client.describe_model_package(ModelPackageName=ModelPackageArn)

        ###########

#### 람다 IAM Role

Lambda 함수에는 SageMaker 에서 수행할 잡(예: 엔드포인트를 배포) 에 대한 IAM 역할이 필요합니다.역할 ARN은 람다스텝에서 제공되어야 합니다. 

Lambda 역할에는 최소한의 람다 실행 정책 외에 여기서는 `세이지메이커:크리에이터 모델', `세이지메이커:생성엔드포인트구성`, `세이지메이커:생성엔드포인트'등의 허용하는 정책이 있어야 합니다. 

`iam_helper.py`의 도우미 함수를 사용하여 Lambda 함수 역할을 생성할 수 있습니다.이 역할은 아마존 관리형 정책 (`세이지메이커풀액세스') 을 사용한다는 점에 유의하십시오.이는 AWS IAM 모범 사례에 따라 최소 권한을 가진 IAM 정책으로 대체해야 합니다.

In [10]:
from src.iam_helper import create_lambda_role

lambda_role = create_lambda_role("lambda-deployment-role")
print("lambda_role: \n", lambda_role)

Using ARN from existing role: lambda-deployment-role
lambda_role: 
 arn:aws:iam::057716757052:role/lambda-deployment-role


## 스텝 생성

### (1) 람다 스텝: 모델 버전 상태를 승인 으로 변경

In [11]:
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

import time 

current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
function_name = "sagemaker-lambda-step-approve-model-deployment-" + current_time

print("function_name: \n", function_name)

function_name: 
 sagemaker-lambda-step-approve-model-deployment-08-25-06-23-05


In [12]:
# Lambda helper class can be used to create the Lambda function
func_approve_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_change_model_approval.py",
    handler="iam_change_model_approval.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_approve_lambda = LambdaStep(
    name="LambdaApproveModelStep",
    lambda_func=func_approve_model,
    inputs={
        "model_package_group_name" : model_package_group_name,
        "ModelApprovalStatus": "Approved",
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

### (2) 세이지 메이커 모델 스텝 생성
- 아래 두 파리미터의 입력이 이전 스텝의 결과가 제공됩니다.
    - image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    - model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,



모델 레지스트리의 해당 모델 패키지 그룹에서 최신 버전 모델에 대한 '추론 도커 이미지', '모델 아티펙트 경로' 를 세이지 메이커 모델 생성시에 입력으로 제공 합니다.

In [13]:
print("image_uri_approved: ", image_uri_approved)
print("ModelDataUrl_approved: ", ModelDataUrl_approved)

image_uri_approved:  366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3
ModelDataUrl_approved:  s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/training_jobs/1hzhiaghayc0-HPTunin-jBvIJGSXWm-001-493861ef/output/model.tar.gz


In [14]:
from sagemaker.model import Model
    
model = Model(
    image_uri= image_uri_approved,
    model_data= ModelDataUrl_approved,    
    sagemaker_session=sagemaker_session,
    role=role,
)

In [15]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    # accelerator_type="ml.eia1.medium",
)
step_create_model = CreateModelStep(
    name="CreateFraudhModel",
    model=model,
    inputs=inputs,
)
step_create_model.add_depends_on([step_approve_lambda]) # step_approve_lambda 완료 후 실행 함.

### (3) 람다 스텝: 엔드포인트 배포

In [16]:
# model_name = project_prefix + "-lambda-model" + current_time
endpoint_config_name = "lambda-deploy-endpoint-config-" + current_time
endpoint_name = "lambda-deploy-endpoint-" + current_time

function_name = "sagemaker-lambda-step-endpoint-deploy-" + current_time

# print("model_name: \n", model_name)
print("endpoint_config_name: \n", endpoint_config_name)
print("endpoint_config_name: \n", len(endpoint_config_name))
print("endpoint_name: \n", endpoint_name)
print("function_name: \n", function_name)




endpoint_config_name: 
 lambda-deploy-endpoint-config-08-25-06-23-05
endpoint_config_name: 
 44
endpoint_name: 
 lambda-deploy-endpoint-08-25-06-23-05
function_name: 
 sagemaker-lambda-step-endpoint-deploy-08-25-06-23-05


In [17]:
# Lambda helper class can be used to create the Lambda function
func_deploy_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_create_endpoint.py",
    handler="iam_create_endpoint.lambda_handler",
    timeout = 900, # 디폴트는 120초 임. 10분으로 연장
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_deploy_lambda = LambdaStep(
    name="LambdaDeployStep",
    lambda_func=func_deploy_model,
    inputs={
        "model_name": step_create_model.properties.ModelName,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

## 모델 빌딩 파이프라인 변수 생성



In [18]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

## 모델 빌딩 파이프라인 정의

In [19]:
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig

pipeline_name = project_prefix + "-Lambda-step"

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        model_approval_status,        
    ],    
    pipeline_experiment_config=PipelineExperimentConfig(
      ExecutionVariables.PIPELINE_NAME,
      ExecutionVariables.PIPELINE_EXECUTION_ID
    ),    
    steps=[step_approve_lambda, step_create_model, step_deploy_lambda],    
    sagemaker_session=sagemaker_session,    
)

In [20]:
import json

definition = json.loads(pipeline.definition())
# definition

## 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [21]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()


In [22]:
execution.wait()

In [23]:
execution.list_steps()

[{'StepName': 'LambdaDeployStep',
  'StartTime': datetime.datetime(2021, 8, 25, 6, 23, 12, 705000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 25, 6, 23, 15, 8000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {}},
 {'StepName': 'CreateFraudhModel',
  'StartTime': datetime.datetime(2021, 8, 25, 6, 23, 11, 708000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 25, 6, 23, 12, 546000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model/pipelines-e1xqc2bx327q-createfraudhmodel-eslwkjhe7s'}}},
 {'StepName': 'LambdaApproveModelStep',
  'StartTime': datetime.datetime(2021, 8, 25, 6, 23, 7, 847000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 25, 6, 23, 11, 458000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {}}]

# 3. 리소스 정리

#### 파이프라인 삭제

- 위에서 생성한 파이프라인을 제거 합니다.
- isDeletePipeline=False, verbose=Fasle
    - 파이프라인을 지우지 않고, 존재하는지 확인 합니다.
- isDeletePipeline=False, verbose=True
    - 파이프라인의 정의를 자세하 확인 합니다.
- isDeletePipeline=True, verbose=True or False
    - 파이프라인을 삭제 합니다.

In [24]:
from src.p_utils import clean_pipeline

# clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=False, verbose=False)   
clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=True, verbose=False)   

pipeline sagemaker-pipeline-step-by-step-phase01-Lambda-step exists
pipeline sagemaker-pipeline-step-by-step-phase01-Lambda-step is deleted


#### 람다 함수 삭제

In [25]:
# Delete the Lambda function
func_deploy_model.delete()
func_approve_model.delete()

{'ResponseMetadata': {'RequestId': 'bad5051a-01cb-4254-8a58-7f1ae345febe',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Wed, 25 Aug 2021 06:23:37 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bad5051a-01cb-4254-8a58-7f1ae345febe'},
  'RetryAttempts': 0}}

### 앤드포인트 컨피그 및 앤드포인트 삭제
- 위의 파이프라인 스탬에서 Async 로 엔드포인트 생성을 요청함. 그래서 아래 엔드포인트 삭제시에 앤드포인트가 생성된 후에 삭제 
함.
- [알림] `An exception occurred: list index out of range` 메세제지가 출력이 되면 해당 앤드포인트가 존재하지 않으니 중단해주세요.

In [26]:
from src.p_utils import is_available_endpoint

while not is_available_endpoint(endpoint_name):
    time.sleep(30)
    print("Endpoint is being creating")

sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_endpoint(EndpointName=endpoint_name)
print("endpoint is deleted")    

Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
Endpoint is being creating
endpoint is deleted
